<a href="https://colab.research.google.com/github/RatanakamonS/DADS6003-2024-S_RTNKMN/blob/main/HW2(DADS6003)_6620422002_Ratanakamon_Somklang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##6620422002 Ratanakamon Somklang

##1. Implement SGD, Mini-Batch from https://github.com/ekaratnida/Applied-machine-learning/blob/master/Week03-MLR/Lab3.ipynb

In [1]:
import numpy as np

In [9]:
x = np.array([[2, 3, 6, 2], [5, 9, 7, 2], [1, 4, 2, 1], [8, 5, 3, 5]])  # x1, x2, x3, x4
y = np.array([12, 9, 6, 7]).reshape(-1, 1)  # y

## 1.1 Stochastic Gradient Descent (sampling: N=1)

In [3]:
def cost_function(theta, x, y):
    y_hat = x.dot(theta)
    c = (1/len(y)) * np.sum((y_hat - y) ** 2)
    return c

In [4]:
def SGD(alpha, x, y, ep=0.001, max_iter=10000):
    converged = False
    iter = 0
    N = x.shape[0]  # จำนวนตัวอย่าง (แนว row)
    theta = np.random.random((x.shape[1], 1))  # ค่า theta เริ่มต้นด้วยค่า theta แบบสุ่ม

    # ทำวนซ้ำเรื่อยๆ จะหยุดการทำงานเมื่อ (1)ค่า J < ค่า ep ที่กำหนด หรือ(2)จำนวนทำซ้ำ >= max_iter
    while not converged:
        for i in range(N):
          #เลือกทุกคอลัมน์ในแถวที่ i ของเมทริกซ์หรืออาร์เรย์ x
          #และ.reshape เพื่อเปลี่ยนรูปอาร์เรย์ใหม่(จำนวนแถวใหม่=1, -1 คือ รักษาความยาวหรือคอลัมน์เดิมของอาร์เรย์)
            xi = x[i, :].reshape(1, -1)  # เลือกข้อมูลแถวที่ i ของเมทริกซ์ x และเปลี่ยนรูปอาร์เรย์

          #เลือกค่าของ y ที่ตำแหน่ง i ในอาร์เรย์ y
            yi = y[i]  # เลือกสมาชิกตัวที่ i เมทริกซ์ y

          # คำนวณค่าคาดการณ์(y_hat)จากการคูณเมทริกซ์ระหว่าง xi และ theta
            y_hat = xi.dot(theta)
            diff = y_hat - yi # คำนวณความแตกต่าง diff ระหว่างค่าคาดการณ์(y_hat) และค่าจริง(yi)

            grad = xi.T.dot(diff)
            theta = theta - alpha * grad #อัปเดตค่า theta โดยใช้ gradient descent ในทุกๆตัวอย่าง xi ของชุดข้อมูล

        # คำนวณค่า error หลังจากที่ได้ผ่านการประมวลผลข้อมูลทั้งหมดในชุดข้อมูลหนึ่งรอบ
        J = cost_function(theta, x, y)

        # ตรวจสอบการหยุดทำงาน
        if J < ep or iter >= max_iter:
            converged = True

        iter += 1

    return theta

In [5]:
if __name__ == '__main__':

    # เพิ่มคอลัมน์ของค่า 1 intercept term เข้าไปในเมทริกซ์
    x_b = np.c_[np.ones((x.shape[0], 1)), x]

    # Learning rate
    alpha = 0.01

    # Train the model โดยใช้ SGD
    theta = SGD(alpha, x_b, y, ep=0.001, max_iter=10000)
    print("Theta = ", theta)

    # ลองแทนค่า x ทดสอบ ใน model เพื่อหาค่าพยากรณ์
    xtest = np.array([[4, 9, 2, 3]])
    xtest_b = np.c_[np.ones((xtest.shape[0], 1)), xtest]
    y_p = xtest_b.dot(theta)
    print("y predict = ", y_p)

Theta =  [[ 2.31201779]
 [-1.44855538]
 [ 0.02183636]
 [ 1.25737739]
 [ 2.47931444]]
y predict =  [[6.6670216]]


##1.2 Mini-Batch Gradient Descent (sampling:1 < n < N)

In [6]:
def cost_function(theta, x, y):
    y_hat = x.dot(theta)
    c = (1/len(y)) * np.sum((y_hat - y) ** 2)
    return c

In [7]:
def Mini_Batch_GD(alpha, x, y, batch_size=20, ep=0.001, max_iter=10000):
    converged = False
    iter = 0
    N = x.shape[0]  # จำนวนตัวอย่าง (แนว row)
    theta = np.random.random((x.shape[1], 1))  # ค่า theta เริ่มต้นด้วยค่า theta แบบสุ่ม

    while not converged:
        # สุ่มลำดับข้อมูลในชุดข้อมูล x และค่าผลลัพธ์ y โดยการจัดเรียงข้อมูลใหม่
        index = np.arange(N)     # สร้างอาร์เรย์ที่ประกอบด้วยตัวเลขเรียงลำดับจาก 0 ถึง N-1
        np.random.shuffle(index) # สุ่มลำดับของตัวเลขในอาร์เรย์ index
        x_shuffled = x[index]    # สร้างเมทริกซ์ x_shuffled ที่มีลำดับแถวที่สอดคล้องกับลำดับใน index ที่ถูกสุ่ม
        y_shuffled = y[index]

        for start in range(0, N, batch_size): # start=0,stop=N,step=batch_size
            end = min(start + batch_size, N)  # stop เวลาใช้สุ่มใหม่ #ใช้ min() เพื่อจำกัดไม่ให้เกิน N
            x_batch = x_shuffled[start:end]   # ข้อมูลที่ถูกสุ่มจัดเรียงใหม่
            y_batch = y_shuffled[start:end]

            # คำนวณตามสูตร
            y_hat = x_batch.dot(theta)
            diff = y_hat - y_batch

            grad = x_batch.T.dot(diff) / len(y_batch)
            theta = theta - alpha * grad

        # คำนวณค่า error หลังจากที่ได้ผ่านการประมวลผลข้อมูลทั้งหมดในชุดข้อมูลหนึ่งรอบ
        J = cost_function(theta, x, y)

        # ตรวจสอบการหยุดทำงาน
        if J < ep or iter >= max_iter:
            converged = True

        iter += 1

    return theta

In [8]:
if __name__ == '__main__':

    # เพิ่มคอลัมน์ของค่า 1 intercept term เข้าไปในเมทริกซ์
    x_b = np.c_[np.ones((x.shape[0], 1)), x]

    # Learning rate
    alpha = 0.01

    # Mini-batch size
    batch_size = 2

    # Train the model โดยใช้ Mini-batch Gradient Descent
    theta = Mini_Batch_GD(alpha, x_b, y, batch_size=batch_size, ep=0.001, max_iter=10000)
    print("Theta = ", theta)

    # ลองแทนค่า x ทดสอบ ใน model เพื่อหาค่าพยากรณ์
    xtest = np.array([[4, 9, 2, 3]])
    xtest_b = np.c_[np.ones((xtest.shape[0], 1)), xtest]
    y_p = xtest_b.dot(theta)
    print("y predict = ", y_p)

Theta =  [[ 2.40240331]
 [-1.41439332]
 [ 0.0038843 ]
 [ 1.25992938]
 [ 2.42324138]]
y predict =  [[6.56937159]]
